Bootstrap Routes


In [1]:
# | default_exp routes.bootstrap

In [2]:
# | exporti
import httpx

import domolibrary.client.DomoError as de
import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

In [3]:
#|hide 
from nbdev.showdoc import show_doc

In [4]:
#| export
from domolibrary.client.DomoAuth import InvalidAuthTypeError

# Error Classes

In [5]:
#| export
class Bootstrap_RetrievalError(de.DomoError):
    def __init__(self, status, response, domo_instance, parent_class, function_name):
        super().__init__(
            status=status,
            message=response,
            domo_instance=domo_instance,
            parent_class=parent_class,
            function_name=function_name,
        )

In [6]:
Bootstrap_RetrievalError(
    status = 400,
    response = 'fail',
    domo_instance = 'test',
    parent_class = 123,
    function_name = 'foo'
)

__main__.Bootstrap_RetrievalError('🛑  Bootstrap_RetrievalError 🛑 - function: 123.foo || status 400 || fail at test')

# Get Bootstrap
The base API requires `DomoFullAuth` flow to query and retrieve data

In [7]:
# | export
@gd.route_function
async def get_bootstrap(
    auth: dmda.DomoFullAuth,  ## only works with DomoFullAuth authentication, do not use TokenAuth
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    parent_class=None,
    debug_num_stacks_to_drop=1,
) -> rgd.ResponseGetData:
    """get bootstrap data"""

    dmda.test_is_full_auth(auth, num_stacks_to_drop=1)

    # url = f"https://{auth.domo_instance}.domo.com/api/domoweb/bootstrap?v2Navigation=false"
    url = (
        f"https://{auth.domo_instance}.domo.com/api/domoweb/bootstrap?v2Navigation=true"
    )

    res = await gd.get_data(
        url=url,
        method="GET",
        auth=auth,
        debug_api=debug_api,
        session=session,
        is_follow_redirects=True,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if not res.is_success:
        raise Bootstrap_RetrievalError(
            status=res.status,
            response=res.response,
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    if res.response == "":
        raise Bootstrap_RetrievalError(
            response="BSR_Features:  no features returned - is there a VPN?",
            status=res.status,
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res

#### sample implementation of get_boostrap

The bootstrap API requires `dmda.DomoFUllAuth`.  Passing Token Auth will throw an error

In [8]:
import os
import domolibrary.client.DomoAuth as dmda


auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

try:
    await get_bootstrap(auth=auth, debug_api=False)

except InvalidAuthTypeError as e:
    print(e)

🛑  InvalidAuthTypeError 🛑 - function: get_bootstrap || This API rquires DomoFullAuth at domo-community


In [9]:
import os

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_password=os.environ["DOJO_PASSWORD"],
    domo_username=os.environ["DOMO_USERNAME"],
)

res = await get_bootstrap(auth=full_auth, debug_api=False)
res.response.keys()

dict_keys(['notifierConfig', 'currentUser', 'data'])

# Get Customer

In [10]:
# |export
@gd.route_function
async def get_bootstrap_customerid(
    auth: dmda.DomoFullAuth,  # this function requires the DomoFullAuth object to authenticate the bootstrap
    session: httpx.AsyncClient = None,  # optional parameter to improve same instance query performance
    debug_api: bool = False,  # pass True to see the parameters sent to the Domo API
    return_raw: bool = False,  # pass True to return the raw API response
    debug_num_stacks_to_drop=2,  # number frames to drop off the stacktrace.  retrieved from `res.traceback_details`
    parent_class: str = None,  # Optional parent class that calls the route function
) -> rgd.ResponseGetData:  # the response contains the string representation of the customer_id
    """retrieves the domo_instance customer id"""

    res = await get_bootstrap(
        auth=auth,
        session=session,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if return_raw:
        return res

    res.response = res.response.get("currentUser").get("USER_GROUP")
    return res

In [11]:
show_doc(get_bootstrap_customerid)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/routes/bootstrap.py#L80){target="_blank" style="float:right; font-size:smaller"}

### get_bootstrap_customerid

>      get_bootstrap_customerid (auth:domolibrary.client.DomoAuth.DomoFullAuth,
>                                session:httpx.AsyncClient=None,
>                                debug_api:bool=False, return_raw:bool=False,
>                                debug_num_stacks_to_drop=2,
>                                parent_class:str=None)

retrieves the domo_instance customer id

#### sample implementation of get_customer_id

In [12]:
import os
import pandas as pd

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_password=os.environ["DOJO_PASSWORD"],
    domo_username=os.environ["DOMO_USERNAME"],
)

await get_bootstrap_customerid(auth=full_auth, debug_api=False, return_raw=False)



ResponseGetData(status=200, response='mmmm-0012-0200', is_success=True, parent_class=None)

# Get Bootstrap Features

In [13]:
# | export
@gd.route_function
async def get_bootstrap_features(
    auth: dmda.DomoAuth,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    return_raw: bool = False,
    debug_num_stacks_to_drop=2,
    parent_class=None,
) -> rgd.ResponseGetData:

    res = await get_bootstrap(
        auth=auth,
        session=session,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if return_raw:
        return res

    res.response = res.response.get("data").get("features")
    return res

#### sample implementation of get_bootstrap_features

In [14]:
import os
import pandas as pd

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_password=os.environ["DOJO_PASSWORD"],
    domo_username=os.environ["DOMO_USERNAME"],
)

res = await get_bootstrap_features(auth=full_auth, debug_api=False, return_raw=False)

pd.DataFrame(res.response[0:5])

,id,name,label,description,enabled,purchased,type,environment,owner
0,4,search,,,True,False,STANDARD,,
1,5,launcher,,,True,True,PREMIUM,,
2,9,profile-reminder,,,True,False,STANDARD,,
3,17,enableSwapDatasource,,,True,False,STANDARD,,
4,19,up,,,True,True,STANDARD,,


# Get Is_Account_V2

In [15]:
# | export

@gd.route_function
async def get_bootstrap_features_is_accountsv2_enabled(
    auth: dmda.DomoAuth,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    return_raw: bool = False,
    debug_num_stacks_to_drop=2,
    parent_class=None,
) -> rgd.ResponseGetData:

    res = await get_bootstrap_features(
        auth=auth,
        session=session,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        return_raw = False
    )

    if return_raw:
        return res

    match_accounts_v2 = next(
        (
            domo_feature
            for domo_feature in res.response
            if domo_feature.get('name') == "accounts-v2"
        ),
        None,
    )

    res.response = True if  match_accounts_v2 else False
    return res

#### sample implementation of get_bootstrap_features_is_accountsv2_enabled

In [16]:
import os
import pandas as pd

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_password=os.environ["DOJO_PASSWORD"],
    domo_username=os.environ["DOMO_USERNAME"],
)

await get_bootstrap_features_is_accountsv2_enabled(auth=full_auth, debug_api=False, return_raw=False)


ResponseGetData(status=200, response=False, is_success=True, parent_class=None)

# Get Bootstrap Pages

In [17]:
# | export
@gd.route_function
async def get_bootstrap_pages(
    auth: dmda.DomoAuth,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    return_raw: bool = False,
    debug_num_stacks_to_drop=2,
    parent_class=None,
) -> rgd.ResponseGetData:
    """this API will return the downstream (children) hierarchy of a page"""
    res = await get_bootstrap(
        auth=auth,
        session=session,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if return_raw:
        return res

    res.response = res.response.get("data").get("pages")
    return res

#### sample get bootstrap pages

In [18]:
import os
import pandas as pd

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_password=os.environ["DOJO_PASSWORD"],
    domo_username=os.environ["DOMO_USERNAME"],
)

res = await get_bootstrap_pages(auth=full_auth, debug_api=False, return_raw=False)

pd.DataFrame(res.response)[0:5]

,id,title,locked,sharedView,pageVisible,virtualPage,isValidBadgePage,canAddCard,bibHiddenPage,hasAccess,hierarchy,separator,type,pageLocation,owner,owners,isOwner,childCount,children
0,-100000,Overview,False,False,True,True,True,True,False,True,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-100003,Favorites,False,False,True,True,NaN,False,False,True,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-100002,Shared,False,False,True,True,NaN,False,False,True,0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30507758,Welcome,True,True,True,NaN,NaN,True,False,True,0,NaN,page,0.0,587894148.0,"[{'id': 587894148, 'type': 'USER', 'displayNam...",True,0.0,NaN
4,1267639476,Community Solutions,True,True,True,NaN,NaN,True,False,True,0,NaN,page,1.0,587894148.0,"[{'id': 587894148, 'type': 'USER', 'displayNam...",False,3.0,"[{'id': '1759878295', 'title': 'Beast Modes', ..."


In [19]:
# | hide
import nbdev

nbdev.nbdev_export()
# !nbqa black bootstrap.ipynb